#  The evaluation of classifiers could has been done on various performance metrics and CPU time. To measure the generalized performance of the algorithms, holdout cross-validation techniques could have been used. For this purpose, training and test samples could have been randomly divided into 9 different sets containing training sizes {10, 20, 30, 40, 50, 60, 70, 80, 90} % of original data and vice versa for test size for 100 times. Then the performance matrices along with CPU time for each holdout proportion could have been calculated.

In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor 
#import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import lime
import lime.lime_tabular

In [40]:
df = pd.read_csv('~/Desktop/junk 3/cervical/cervicaData.csv')
df.head()

,Age,Numberofsexualpartners,Firstsexualintercourse,Numofpregnancies,Smokes,Smokesyears,Smokespacksyear,HormonalContraceptives,HormonalContraceptivesyears,IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,DxCancer,DxCIN,DxHPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4,15,1,0,0,0,0,0,0,...,?,?,0,0,0,0,0,0,0,0
1,15,1,14,1,0,0,0,0,0,0,...,?,?,0,0,0,0,0,0,0,0
2,34,1,?,1,0,0,0,0,0,0,...,?,?,0,0,0,0,0,0,0,0
3,52,5,16,4,1,37,37,1,3,0,...,?,?,1,0,1,0,0,0,0,0
4,46,3,21,4,0,0,0,1,15,0,...,?,?,0,0,0,0,0,0,0,0


In [12]:
#df['Cancer']=df["Hinselmann"] + df["Schiller"] + df["Citology"] + df["Biopsy"]
#df.drop(["Hinselmann","Schiller","Citology","Biopsy"], axis = 1)

In [13]:
#df.isna().all()

In [16]:
    #https://www.kaggle.com/rushirajchavan/riskfactors
    
    # remove column having more than 70% missing values.
    df = df.loc[:,df.columns[df.isnull().sum() / df.shape[0] < 0.9 ]]
    
    # fill missing values with median values
    for col in df.columns[df.isnull().any()]:
        df[col].fillna(df[col].median(),inplace=True)
    

SyntaxError: invalid syntax (<ipython-input-16-d6aee54bf2a7>, line 1)

In [41]:
df = df.replace('?', np.NaN)


In [37]:
df= df.loc[:,df.columns[df.isnull().sum() / df.shape[0] < 0.9 ]]

print(df)

     Age Numberofsexualpartners Firstsexualintercourse Numofpregnancies  \
0     18                      4                     15                1   
1     15                      1                     14                1   
2     34                      1                    NaN                1   
3     52                      5                     16                4   
4     46                      3                     21                4   
..   ...                    ...                    ...              ...   
853   34                      3                     18                0   
854   32                      2                     19                1   
855   25                      2                     17                0   
856   33                      2                     24                2   
857   29                      2                     20                1   

    Smokes Smokesyears Smokespacksyear HormonalContraceptives  \
0        0           0            

In [42]:
df.dtypes

Age                                  int64
Numberofsexualpartners              object
Firstsexualintercourse              object
Numofpregnancies                    object
Smokes                              object
Smokesyears                         object
Smokespacksyear                     object
HormonalContraceptives              object
HormonalContraceptivesyears         object
IUD                                 object
IUDyears                            object
STDs                                object
STDsnumber                          object
STDscondylomatosis                  object
STDscervicalcondylomatosis          object
STDsvaginalcondylomatosis           object
STDsvulvoperinealcondylomatosis     object
STDssyphilis                        object
STDspelvicinflammatorydisease       object
STDsgenitalherpes                   object
STDsmolluscumcontagiosum            object
STDsAIDS                            object
STDsHIV                             object
STDsHepatit

In [29]:
x_features=list(df.columns)
x_features.remove('Biopsy')
x_features_categorical=['Numberofsexualpartners','Firstsexualintercourse','Numofpregnancies','Smokes','Smokesyears','Smokespacksyear','HormonalContraceptives','HormonalContraceptivesyears','IUD','IUDyears','STDs','STDsnumber','STDscondylomatosis','STDscervicalcondylomatosis','STDsvaginalcondylomatosis','STDsvulvoperinealcondylomatosis','STDssyphilis','STDspelvicinflammatorydisease','STDsgenitalherpes','STDsmolluscumcontagiosum','STDsAIDS','STDsHIV','STDsHepatitisB','STDsHPV']
x_features_numerical=[i for i in x_features if i not in x_features_categorical]
x_features_numerical

['Age',
 'STDsNumberofdiagnosis',
 'DxCancer',
 'DxCIN',
 'DxHPV',
 'Dx',
 'Hinselmann',
 'Schiller',
 'Citology']

In [43]:
df.isnull().sum() / df.shape[0] *100

Age                                  0.000000
Numberofsexualpartners               3.030303
Firstsexualintercourse               0.815851
Numofpregnancies                     6.526807
Smokes                               1.515152
Smokesyears                          1.515152
Smokespacksyear                      1.515152
HormonalContraceptives              12.587413
HormonalContraceptivesyears         12.587413
IUD                                 13.636364
IUDyears                            13.636364
STDs                                12.237762
STDsnumber                          12.237762
STDscondylomatosis                  12.237762
STDscervicalcondylomatosis          12.237762
STDsvaginalcondylomatosis           12.237762
STDsvulvoperinealcondylomatosis     12.237762
STDssyphilis                        12.237762
STDspelvicinflammatorydisease       12.237762
STDsgenitalherpes                   12.237762
STDsmolluscumcontagiosum            12.237762
STDsAIDS                          

In [31]:
df_iud=df.copy()

x_features_categorical.remove('IUD')
for i in x_features_categorical:
    df_iud[i]=df_iud[i].fillna(df_iud[i].mode()[0])
    #print(df_iud[i].mode()[0])
for i in x_features_numerical:
    df_iud[i]=df_iud[i].fillna(df_iud[i].median())
    
df_iud=df_iud.astype('float')
df_iud[x_features_categorical]=df_iud[x_features_categorical].replace(0,'no')
df_iud[x_features_categorical]=df_iud[x_features_categorical].replace(1,'yes')
df_iud=pd.get_dummies(df_iud)

train_iud=df_iud[df_iud.IUD.isnull()==False]
test_iud=df_iud[df_iud.IUD.isnull()]

train_iud_x=train_iud.drop('IUD',axis=1)
train_iud_y=train_iud['IUD']

test_iud_x=test_iud.drop('IUD',axis=1)
test_iud_y=test_iud['IUD']

dt=DecisionTreeClassifier()
iud_model=dt.fit(train_iud_x,train_iud_y)
test_iud['IUD']=iud_model.predict(test_iud_x)

iud_complete=pd.concat([train_iud,test_iud],axis=0)

df_impute=df.copy()
df_impute['IUD']=iud_complete['IUD'].sort_index()

<ipython-input-31-eb7dc59af45c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_iud['IUD']=iud_model.predict(test_iud_x)


In [32]:
df_i=df.copy()

x_features_categorical.remove('IUDyears')
for i in x_features_categorical:
    df_i[i]=df_i[i].fillna(df_i[i].mode()[0])
    #print(df_iud[i].mode()[0])
for i in x_features_numerical:
    df_i[i]=df_i[i].fillna(df_i[i].median())
    
df_i=df_i.astype('float')
df_i[x_features_categorical]=df_i[x_features_categorical].replace(0,'no')
df_i[x_features_categorical]=df_i[x_features_categorical].replace(1,'yes')
df_i=pd.get_dummies(df_i)

train_i=df_i[df_i.IUD.isnull()==False]
test_i=df_i[df_i.IUD.isnull()]

train_i_x=train_i.drop('IUDyears',axis=1)
train_i_y=train_i['IUDyears']

test_i_x=test_i.drop('IUDyears',axis=1)
test_i_y=test_i['IUDyears']

dt=DecisionTreeClassifier()
iud_model=dt.fit(train_i_x,train_i_y)
test_i['IUDyears']=i_model.predict(test_i_x)

i_complete=pd.concat([train_i,test_i],axis=0)

df_impute=df.copy()
df_impute['IUDyears']=i_complete['IUDyears'].sort_index()

ValueError: Unknown label type: 'continuous'

In [34]:
df_hor=df_impute.drop(['Biopsy'],axis=1)

x_features_categorical.remove('HormonalContraceptives')
for i in x_features_categorical:
    df_hor[i]=df_hor[i].fillna(df_hor[i].mode()[0])
for i in x_features_numerical:
    df_hor[i]=df_hor[i].fillna(df_hor[i].median())
    
df_hor=df_hor.astype('float')
df_hor[x_features_categorical]=df_hor[x_features_categorical].replace(0,'no')
df_hor[x_features_categorical]=df_hor[x_features_categorical].replace(1,'yes')
df_hor=pd.get_dummies(df_hor)

train_hor=df_hor[df_hor.HormonalContraceptives.isnull()==False]
test_hor=df_hor[df_hor.HormonalContraceptives.isnull()]

train_hor_x=train_hor.drop('HormonalContraceptives',axis=1)
train_hor_y=train_hor['HormonalContraceptives']

test_hor_x=test_hor.drop('HormonalContraceptives',axis=1)
test_hor_y=test_hor['HormonalContraceptives']

dt=DecisionTreeClassifier()
hor_model=dt.fit(train_hor_x,train_hor_y)
test_hor['HormonalContraceptives']=hor_model.predict(test_hor_x)

hor_complete=pd.concat([train_hor,test_hor],axis=0)

df_impute['HormonalContraceptives']=hor_complete['Hormonal_Contraceptives'].sort_index()

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [54]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,848,849,850,851,852,853,854,855,856,857
Age,18,15,34,52,46,42,51,26,45,44,...,31,32,19,23,43,34,32,25,33,29
Numberofsexualpartners,4,1,1,5,3,3,3,1,1,3,...,3,3,1,2,3,3,2,2,2,2
Firstsexualintercourse,15,14,NaN,16,21,23,17,26,20,15,...,18,18,14,15,17,18,19,17,24,20
Numofpregnancies,1,1,1,4,4,2,6,3,5,NaN,...,1,1,0,2,3,0,1,0,2,1
Smokes,0,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
Smokesyears,0,0,0,37,0,0,34,0,0,1.266972909,...,0,11,0,0,0,0,0,0,0,0
Smokespacksyear,0,0,0,37,0,0,3.4,0,0,2.8,...,0,0.16,0,0,0,0,0,0,0,0
HormonalContraceptives,0,0,0,1,1,0,0,1,0,0,...,1,1,0,0,1,0,1,1,1,1
HormonalContraceptivesyears,0,0,0,3,15,0,0,2,0,0,...,0.5,6,0,0,5,0,8,0.08,0.08,0.5
IUD,0,0,0,0,0,0,1,1,0,NaN,...,0,0,0,0,0,0,0,0,0,0
